### 사용자 정의 데이터셋과 모델과 학습
- iris.csv ==> 사용자 정의 데이터셋
- DNN 모델 ==> 사용자 정의 모델

[1] 모듈 로딩 및 데이터 준비 <hr>

In [1]:
# 모듈 로딩
import torch 
import torch.nn as nn
import torch.nn.functional as F 
import torch.optim as optim
from torchmetrics.classification import F1Score
from torchinfo import summary

from torch.utils.data import Dataset, DataLoader

import pandas as pd
from sklearn.preprocessing import LabelEncoder      # 타겟 컬럼 수치화

In [2]:
# 데이터
DATA_FILE = '../../../Data/iris.csv'

In [3]:
# CSV ==> DataFrame
irisDF = pd.read_csv(DATA_FILE)
irisDF.head(2)

,sepal.length,sepal.width,petal.length,petal.width,variety
0,5.1,3.5,1.4,0.2,Setosa
1,4.9,3.0,1.4,0.2,Setosa


In [4]:
# 타겟 컬럼 수치화 ==> LabelEncoder
encoder = LabelEncoder()
encoder.fit(irisDF['variety'])
irisDF['variety'] = encoder.transform(irisDF['variety'])

In [5]:
irisDF.head(2)

,sepal.length,sepal.width,petal.length,petal.width,variety
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0


[2] 사용자 정의 데이터셋 클래스 생성 <hr>

In [6]:
# ----------------------------------------------------
# 클래스 목적 : 학습용 데이터셋 텐서화 및 전처리
# 클래스 이름 : CustomDataSet
# 부모 클래스 : torch.utils.data.Dataset
# 매개   변수 : featureDF, targetDF
# ----------------------------------------------------

In [7]:
class CustomDataset(Dataset):
    # 데이터 로딩 및 전처리 진행과 인스턴스 생성 메서드
    def __init__(self, featureDF, targetDF):
        super().__init__()
        self.featureDF = featureDF
        self.targetDF = targetDF
        self.n_rows = featureDF.shape[0]
        self.n_features = featureDF.shape[1]

    # 데이터의 개수 반환 메서드
    def __len__(self):
        return self.n_rows

    # 특정 index의 데이터와 타겟 반환 메서드 => Tensor 반환!!!
    def __getitem__(self, idx):
        featureTS = torch.FloatTensor(self.featureDF.iloc[idx].values)
        targetTS = torch.FloatTensor(self.targetDF.iloc[idx].values)
        return featureTS, targetTS

[3] 데이터셋 인스턴스 생성 <hr>

In [8]:
featureDF, targetDF = irisDF[irisDF.columns[:-1]], irisDF[[irisDF.columns[-1]]]

print(f'featureDF => {featureDF.shape}, targetDF => {targetDF.shape}')

featureDF => (150, 4), targetDF => (150, 1)


In [9]:
# IRIS 데이터셋 인스턴스 생성
irisDS = CustomDataset(featureDF, targetDF)

In [10]:
# IRIS 데이터셋 속성
irisDS.n_features, irisDS.n_rows

(4, 150)

In [11]:
irisDS[0]

(tensor([5.1000, 3.5000, 1.4000, 0.2000]), tensor([0.]))

[4] 데이터로더 인스턴스 생성

In [30]:
# 필요한 것 : Dataset 인스턴스, Batch_size=1[기본값]
irisDL = DataLoader(irisDS, batch_size=10)
# 지정한 배치사이즈 만큼 데이터셋에서 꺼내옴

In [31]:
for dataTS, targetTS in irisDL:
    print(dataTS.shape, targetTS.shape)
    break

torch.Size([10, 4]) torch.Size([10, 1])


[5] 모델 준비 <hr>

In [32]:
# -------------------------------------------------------------
# 모델  이름 : CustomModel
# 부모클래스 : nn.Module
# 매개  변수 : None
# 모델  구조
# - 입력층   : 입력 4개,    출력 10개    AF ReLU -> LeakyReLU
# - 은닉층   : 입력 10개,   출력 30개    AF ReLU -> LeakyReLU
# - 출력층   : 입력 30개,   출력  3개    AF 분류 - 다중 softmax
# -------------------------------------------------------------
class CustomModel(nn.Module):
    # 모델 구성 및 인스턴스 생성 메서드
    def __init__(self):
        super().__init__()
        self.in_layer = nn.Linear(4, 10)
        self.hidden_layer = nn.Linear(10, 30)
        self.out_layer = nn.Linear(30, 3) # 붓꽃 종류가 3종류라 출력이 3개

    # 순방향 학습 메서드
    def forward(self, x):
        y = F.relu(self.in_layer(x))
        y = F.relu(self.hidden_layer(y))
        # return F.softmax(self.out_layer(y))
        return self.out_layer(y)

[6] 학습 <hr>

In [33]:
# 모델 인스턴스 생성
model = CustomModel()
print(model)
summary(model)

CustomModel(
  (in_layer): Linear(in_features=4, out_features=10, bias=True)
  (hidden_layer): Linear(in_features=10, out_features=30, bias=True)
  (out_layer): Linear(in_features=30, out_features=3, bias=True)
)


Layer (type:depth-idx)                   Param #
CustomModel                              --
├─Linear: 1-1                            50
├─Linear: 1-2                            330
├─Linear: 1-3                            93
Total params: 473
Trainable params: 473
Non-trainable params: 0

In [41]:
# 배치크기만큼 데이터와 타겟 추출해서 학습 진행
TS_loss, TS_score = [], []
for epoch in range(10):
    loss_total, score_total = 0, 0
    for dataTS, targetTS in irisDL:

        # 배치크기 만큼의 학습 데이터
        print(dataTS.shape, targetTS.shape, targetTS.dtype)
        targetTS = targetTS.reshape(-1).long()

        # 배치크기만큼 학습 진행
        pred_y = model(dataTS)
        print(pred_y.shape, targetTS.reshape(-1).shape)

        # 손실 계산
        loss = nn.CrossEntropyLoss()(pred_y, targetTS)
        loss_total += loss

        score = F1Score()(pred_y, targetTS)
        score_total += score

    TS_loss.append(loss_total/len(irisDL))
    TS_score.append(score_total/len(irisDL))

torch.Size([10, 4]) torch.Size([10, 1]) torch.float32
torch.Size([10, 3]) torch.Size([10])


TypeError: __new__() missing 1 required positional argument: 'task'